
## 恒生指数（HSI）研究

## 摘要

本报告基于恒生指数日度数据，依次完成探索性分析、平稳性检验、ARIMA 建模、GARCH 及其扩展、特征工程与机器学习预测、策略回测、协整/VAR 诊断及指标显著性检验。主要结论如下：

- ADF/KPSS 说明收益率序列平稳；ACF/PACF 与 Ljung-Box 显示存在 ARCH 效应。
- ARIMA(2,0,3) 作为课程基准模型，在 252 日测试窗口的方向准确率约 52%。
- GJR-GARCH 捕捉波动不对称性；进一步引入 S&P500/VIX 等外生变量后，GARCH-X 显著降低 AIC，并量化全球风险偏好的影响。
- 引入技术指标与外部因子后，随机森林与逻辑回归可在方向预测上取得小幅提升，并可衍生简单策略。
- VAR 与 Granger 检验表明 S&P500 对恒指具有显著冲击；指标显著性分析进一步说明 S&P500/VIX 的解释力远优于传统单一技术指标。



In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries loaded successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


## 1. Data Loading and Overview

### 1.1 Load Processed Data


In [ ]:
# Load processed HSI data
df = pd.read_csv('../data/hsi_with_returns.csv', index_col=0, parse_dates=True, header=[0, 1])

# Flatten multi-level columns if needed
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

print(f"Dataset Shape: {df.shape}")
print(f"Time Period: {df.index[0].date()} to {df.index[-1].date()}")
print(f"Total Trading Days: {len(df)}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst 5 rows:")
df.head()


## 2. Exploratory Data Analysis (EDA)

### 2.1 Price Series
z

In [ ]:
# Display price series plot
display(Image(filename='../output/figures/01_price_series.png'))


### 2.2 收益率分布


In [ ]:
# Returns series
display(Image(filename='../output/figures/02_returns_series.png'))

# Distribution analysis
display(Image(filename='../output/figures/03_distribution.png'))


## 3. 平稳性分析

目的：判断序列是否平稳，这是 ARIMA 建模的前提条件。

方法：
- Augmented Dickey-Fuller (ADF) 检验：原假设为存在单位根（序列非平稳）。
- KPSS 检验：原假设为序列平稳。

结果：收益率序列在 ADF 检验下拒绝单位根假设，而 KPSS 检验未拒绝平稳假设，因而可视为平稳序列。


In [ ]:
# Stationarity test results
display(Image(filename='../output/figures/08_stationarity_returns.png'))
display(Image(filename='../output/figures/10_stationarity_summary.png'))


## 4. 序列相关性分析

目的：识别收益率及其平方的自相关结构，为 ARIMA 阶数选择和 ARCH 效应检验提供依据。

主要结论：
- 收益率本身的自相关较弱；
- 平方收益率在多阶滞后上存在显著自相关，说明波动率呈聚集性，可用 ARCH/GARCH 模型刻画。


In [ ]:
# ACF/PACF for returns
display(Image(filename='../output/figures/11_acf_pacf_returns.png'))

# ACF/PACF for squared returns (ARCH effects)
display(Image(filename='../output/figures/14_acf_pacf_squared_returns.png'))


## 5. ARIMA 建模

选定模型：ARIMA(2,0,3)
- p = 2：包含两个自回归项；
- d = 0：收益率已平稳，无需差分；
- q = 3：包含三个移动平均项。

模型依据：通过遍历 (p,d,q) 组合并比较 AIC/BIC，ARIMA(2,0,3) 在测试范围内最优。


In [ ]:
# ARIMA model comparison
display(Image(filename='../output/figures/18_arima_model_comparison.png'))

# ARIMA diagnostics
display(Image(filename='../output/figures/17_arima_diagnostics.png'))


## 6. GARCH 波动率建模

基准模型：GARCH(1,1) + t 分布残差。

建模目的：
- 描述波动率的聚集特征，即高波动期往往连续出现；
- 处理收益率分布的厚尾现象。

模型形式：
$$\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma_{t-1}^2$$


In [ ]:
# GARCH model comparison
display(Image(filename='../output/figures/19_garch_model_comparison.png'))

# Conditional volatility over time
display(Image(filename='../output/figures/20_conditional_volatility.png'))


## 7. 预测与评估

设置：
- 训练区间：2015-01-05 至 2024-11-13，共 2427 个交易日；
- 测试区间：2024-11-14 至 2025-11-21，共 252 个交易日（约 12 个月）；
- 方法：滚动窗口，每次基于历史样本更新模型后预测下一期。

指标：
- MAE（平均绝对误差）；
- RMSE（均方根误差）；
- R²（判定系数）；
- 方向准确率。


In [ ]:
# ARIMA forecast results
display(Image(filename='../output/figures/22_arima_forecast.png'))

# GARCH volatility forecast
display(Image(filename='../output/figures/23_garch_volatility_forecast.png'))


### 7.1 模型比较结果


In [ ]:
# Load and display forecast comparison
comparison = pd.read_csv('../output/forecast_comparison.csv')
print("Forecast Method Comparison:\n")
print(comparison.to_string(index=False))

# Display comparison plot
display(Image(filename='../output/figures/24_forecast_comparison.png'))


## 9. 特征扩展与机器学习预测

我们基于 `scripts/07_feature_engineering.py` 构建了包含技术指标与外部市场因子的特征集，并使用 `scripts/08_ml_models.py` 训练机器学习模型；随后使用 `scripts/09_backtest.py` 对预测信号进行简易策略评估。下文汇总关键结果。


In [ ]:
import json
with open('../output/feature_metadata.json', 'r', encoding='utf-8') as f:
    meta = json.load(f)

pd.DataFrame({
    '统计项': ['样本量', '起止日期', '特征数量', '目标列', '外部市场因子'],
    '数值': [
        meta['total_rows'],
        f"{meta['start_date']} ~ {meta['end_date']}",
        meta['feature_count'],
        ', '.join(meta['targets']),
        ', '.join(meta['external_features'])
    ]
})


### 9.1 模型性能对比
- **回归目标（预测下一日收益）**：采用 Ridge 与 Random Forest 回归。后者在测试期取得 R²≈0.046、方向准确率 57%，优于原始 ARIMA (R²≈0.037、方向 50%)。
- **方向分类**：逻辑回归（带 class-weight）准确率 58%，召回率 79%，能够更好地捕捉上涨日；梯度提升作为非线性基准。

下表直接读取 `output/ml_*.csv`：


In [ ]:
reg_metrics = pd.read_csv('../output/ml_regression_metrics.csv')
cls_metrics = pd.read_csv('../output/ml_classification_metrics.csv')

print("回归模型：")
display(reg_metrics)
print("\n分类模型：")
display(cls_metrics)


### 9.2 特征重要性
Random Forest 的重要性排序体现：
- **SP500 / VIX 变动**：全球风险偏好对恒指有显著影响；
- **长周期移动均线偏离度**：反映趋势位置；
- **成交量/波动率指标**：与市场动能相关。



In [ ]:
feature_imp = pd.read_csv('../output/ml_feature_importance.csv')
feature_imp.head(15)


### 9.3 策略回测（150 日测试期）
- **Regression_RF_Targeted**：仅在预测收益绝对值超过 0.15% 时开仓，年化收益 49%、Sharpe≈3.88，说明“择机交易”优于无差别持仓。
- **Logistic_Prob**：基于概率阈值的多空切换，Sharpe≈2.73。
- 相比 Buy & Hold（Sharpe≈1.60），新模型在风险调整收益上有显著提升。


In [ ]:
backtest = pd.read_csv('../output/backtest_summary.csv')
display(backtest)

display(Image(filename='../output/figures/25_strategy_equity.png'))


## 10. 多变量协整与 VAR 分析

我们选取 HSI、HSCEI、USD/HKD、S&P500 四个序列进行联合建模，步骤如下：
1. 对每个序列进行 ADF 检验，确认它们在一阶差分后平稳；
2. 执行 Johansen 检验验证协整关系；
3. 在差分数据上拟合 VAR 模型，分析 Granger 因果与冲击响应。



In [ ]:
var_adf = pd.read_csv('../output/var_adf_results.csv')
var_johansen = pd.read_csv('../output/var_johansen_trace.csv')

print('ADF 检验结果 (水平 & 一阶差分):')
display(var_adf)
print('\nJohansen Trace Test:')
display(var_johansen)



**解读**：水平序列大多非平稳，但一阶差分显著平稳；Johansen Trace 统计量均低于 5% 临界值，说明在样本期内协整关系并不显著，符合市场弱有效的假设。不过，差分后的 VAR 仍可用于考察短期冲击传递。


In [ ]:
var_granger = pd.read_csv('../output/var_granger_results.csv')
display(var_granger)

display(Image(filename='../output/figures/26_var_irf.png'))
display(Image(filename='../output/figures/27_var_fevd.png'))


- **Granger 因果**：S&P500 → HSI 的因果关系在 1% 水平显著，而 USD/HKD、HSCEI 对 HSI 的短期影响较弱；
- **冲击响应**：美元区/美股冲击对恒指存在短期正向响应，约 5-7 日逐渐消散；
- **方差分解**：恒指自身波动贡献最大，但 S&P500 冲击占比逐步上升，说明全球风险情绪是重要解释变量。


## 11. GARCH 扩展模型

进一步比较 **EGARCH** / **GJR-GARCH** 与基准 GARCH(1,1)，以检验杠杆效应与不对称波动：


In [ ]:
garch_ext = pd.read_csv('../output/garch_extension_comparison.csv')
display(garch_ext)

display(Image(filename='../output/figures/28_egarch_volatility.png'))
display(Image(filename='../output/figures/29_egarch_residuals.png'))


- **模型比较**：GJR-GARCH 取得最低 AIC/BIC，说明考虑负向冲击的不对称效应可提升拟合；
- **持久性**：GJR 的 α+β≈0.95，表明波动率仍高度持久，但低于标准 GARCH 的 0.99；
- **残差诊断**：EGARCH/GJR 均显著减弱平方残差的自相关，说明对极端波动的刻画更合理。


## 12. GARCH-X 外生驱动分析

- 在均值方程中同时引入价格技术指标（SMA_ratio_20、RSI_14）与外部市场因子（S&P500、VIX、USDHKD），评估外生变量对恒指收益及波动率的解释力；
- 对照 GARCH(1,1) 与 GARCH-X 的对数似然、AIC/BIC 及持久性，判断课程讨论的“全球风险偏好影响恒指”命题；
- 重点关注 S&P500 与 VIX 系数是否显著为零，以验证外部冲击的即时影响。


In [ ]:
import pandas as pd
from IPython.display import Image, display

gx_comp = pd.read_csv('../output/garch_x_results.csv')
gx_coef = pd.read_csv('../output/garch_x_coefficients.csv')
print('GARCH vs. GARCH-X 指标：')
display(gx_comp)
print('\nGARCH-X 外生变量系数（前 10 项）：')
display(gx_coef.head(10))

display(Image(filename='../output/figures/30_garchx_volatility.png'))


## 13. 技术指标显著性检验

- 选取 SMA 偏离、RSI、MACD、布林通道百分位以及 S&P500/VIX 收益，使用 OLS 回归、Ljung-Box 与 Granger 检验其对下一期收益的预测力；
- 指标若在 t 检验中显著，且 Granger p 值较低，则说明其具备可解释性；否则可视为噪声，符合弱式有效市场假说。


In [ ]:
ind_df = pd.read_csv('../output/indicator_predictability.csv')
print(ind_df[['Indicator', 'Description', 'tStat', 'PValue', 'DirectionAcc', 'Granger_p']])

display(Image(filename='../output/figures/31_indicator_tstats.png'))


## 14. 结论与讨论

### 14.1 主要发现

- ARIMA(2,0,3) 在 252 个测试交易日上的 R² 为 0.018，方向准确率约 52%，仍符合弱式有效市场假设提出的“价格变动接近随机”的特征，可作为课程要求的基准模型。
- GARCH 家族中，GJR-GARCH(1,1) 取得最低 AIC/BIC，并揭示负向冲击会放大波动；GARCH-X 通过外生变量进一步降低 AIC 至 7488，S&P500/VIX 系数在 1% 水平显著，量化了外部冲击的作用。
- VAR 分析虽未发现显著协整，但表明 S&P500 对恒生指数具有显著的 Granger 因果关系，冲击响应与方差分解量化了全球风险情绪对恒指的短期影响。
- 机器学习与特征工程作为补充手段，随机森林和逻辑回归在方向预测上仅取得小幅提升；在无交易成本假设下，基于概率阈值的策略可以获得较高的 Sharpe 值，但仍需警惕过拟合与现实约束。
- 指标显著性检验显示，多数单一技术指标的 t 统计不显著，而 S&P500/VIX 的统计量绝对值大于 13，说明外部因子是改善预测能力的关键来源。

### 14.2 理论含义

- 结果支持弱式有效市场假设：仅依靠历史价格难以获得稳定超额收益。
- 波动率具有持久性与不对称性，因而需要 GARCH 扩展模型来描述负向冲击的杠杆效应；同时，GARCH-X 证明外生冲击可直接进入均值方程。
- 多资产 VAR 与指标 Granger 检验提供了度量全球市场冲击的工具，即使不存在长期协整，短期冲击仍可通过 IRF/FEVD 与预测检验捕捉。

### 14.3 综合评价

本项目构建了完整的时间序列分析流程：EDA → 平稳性检验 → ARIMA → GARCH/GJR/GARCH-X → 特征工程 → 机器学习与策略回测 → VAR/协整诊断 → 指标显著性检验。各阶段的诊断结果、图表和数据文件均已整理，可在 `notebooks/main_analysis.ipynb` 中一览。
